## MLflow in Azure Databricks

### What is MLflow?
- An open-source platform for the machine learning lifecycle
- Commonly used as part of a machine learning operations (ML Ops) solution
- Integrated into many commonly used Machine Learning services, including:
  - Azure Machine Learning
  - Microsoft Fabric
  - Azure Databricks


### Mlflow capabilities

**MLflow Tracking**
- Log parameters, metrics, and other data when training and evaluating models
- View and compare metrics from multiple previously run experiments

**MLflow Projects**
- Package code for consistent deployment and reproducibility
- Define reusable Python code environments

**MLflow Models**
- Package models for distribution and deployment
- Support for common formats (Scikit-Learn, Spark MLLib, PyTorch TensorFlow, …)

**MLflow Model Registry**
- Register and manage models, including versioning
- Serve models for inferencing


### Running experiments with MLflow
MLflow experiments allow data scientists to track training runs in a collection called an experiment. Experiment runs are useful for comparing changes over time or comparing the relative performance of models with different hyperparameter values.

Creating an experiment in Azure Databricks happens automatically when you start a run. In this case, the experiment's name is the name of the notebook. It's possible to export a variable named MLFLOW_EXPERIMENT_NAME to change the name of your experiment should you choose.

You can use Mlflow methods to explicitly log:
- Parameters
- Metrics
- Models

In the Azure Databricks portal, the Experiments page enables you to view details of each experiment run; including logged values for parameters, metrics, and other artifacts.


### Registering and serving models with MLflow

Model registration in MLflow helps you track models for two main reasons:

- It lets you serve models easily for real-time, streaming, or batch predictions without writing extra code—MLflow creates REST APIs or batch scoring methods automatically.
- It allows you to version models over time, so you can track changes and compare past versions.

When you train a model, you can log it as part of the experiment run, including performance metrics.

- You can register the model later via the experiment UI.
- Or, you can register automatically by adding registered_model_name to log_model().

Inferencing (predicting labels for new data) can be done in Azure Databricks by:

- Hosting the model as a real-time REST API.
- Performing streaming predictions on delta tables.
- Running batch predictions and saving results to folders.

You can deploy models directly from the Models page in Databricks.

## Use MLflow in Azure Databricks

### Ingest and prepare data

In [0]:
%run ../PySpark/DatasetSourcePath

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

path = sourcePath + "/dataset/penguins.csv"   
data = spark.read.format("csv").option("header", "true").load(path)
data = data.dropna().select(col("Island").astype("string"),
                            col("CulmenLength").astype("float"),
                            col("CulmenDepth").astype("float"),
                            col("FlipperLength").astype("float"),
                            col("BodyMass").astype("float"),
                            col("Species").astype("int")
                          )
display(data.sample(0.2))
   
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
print ("Training Rows:", train.count(), " Testing Rows:", test.count())

### Run an MLflow experiment

- MLflow lets you track model training runs and log evaluation metrics.
- Recording these details helps you iterate and improve your model effectively.
- You can keep using your usual libraries (like Spark MLlib) for training and evaluation.
- Just add MLflow commands to log metrics and information while training the model.

###  Create a function

- In ML projects, you often train models with different parameters and log results.
- To make this easier, you can create a function that wraps the training process.
- Then, simply call the function with different parameters to test and compare models.

In [0]:
def train_penguin_model(training_data, test_data, maxIterations, regularization):
    import mlflow
    import mlflow.spark
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    import time
   
    # Start an MLflow run
    with mlflow.start_run():
   
        catFeature = "Island"
        numFeatures = ["CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass"]
   
        # Define the feature engineering and model steps
        catIndexer = StringIndexer(inputCol=catFeature, outputCol=catFeature + "Idx")
        numVector = VectorAssembler(inputCols=numFeatures, outputCol="numericFeatures")
        numScaler = MinMaxScaler(inputCol = numVector.getOutputCol(), outputCol="normalizedFeatures")
        featureVector = VectorAssembler(inputCols=["IslandIdx", "normalizedFeatures"], outputCol="Features")
        algo = LogisticRegression(labelCol="Species", featuresCol="Features", maxIter=maxIterations, regParam=regularization)
   
        # Chain the steps as stages in a pipeline
        pipeline = Pipeline(stages=[catIndexer, numVector, numScaler, featureVector, algo])
   
        # Log training parameter values
        print ("Training Logistic Regression model...")
        mlflow.log_param('maxIter', algo.getMaxIter())
        mlflow.log_param('regParam', algo.getRegParam())
        model = pipeline.fit(training_data)
   
        # Evaluate the model and log metrics
        prediction = model.transform(test_data)
        metrics = ["accuracy", "weightedRecall", "weightedPrecision"]
        for metric in metrics:
            evaluator = MulticlassClassificationEvaluator(labelCol="Species", predictionCol="prediction", metricName=metric)
            metricValue = evaluator.evaluate(prediction)
            print("%s: %s" % (metric, metricValue))
            mlflow.log_metric(metric, metricValue)
   
   
        # Log the model itself
        unique_model_name = "classifier-" + str(time.time())
        mlflow.spark.log_model(model, unique_model_name, mlflow.spark.get_default_conda_env())
        modelpath = "/model1/%s" % (unique_model_name)
        mlflow.spark.save_model(model, modelpath)
   
        print("Experiment run complete.")

In [0]:
train_penguin_model(train, test, 10, 0.2)

### Register and deploy a model with MLflow

- Besides tracking training runs, MLflow helps you manage trained models.
- Each experiment run already logs the model you trained.
- You can also register models and deploy them so they’re ready to serve predictions to client applications.

1. View the details page for the most recent experiment run.
1. Use the **Register Model** button to register the model that was logged in that experiment and when prompted, create a new model named **Penguin Predictor**.
1. When the model has been registered, view the **Models** page (in the navigation bar on the left) and select the **Penguin Predictor** model.
1. In the page for the **Penguin Predictor** model, use the **Use model for inference** button to create a new real-time endpoint with the following settings:
    - **Model**: Penguin Predictor
    - **Model version**: 1
    - **Endpoint**: predict-penguin
    - **Compute size**: Small

    The serving endpoint is hosted in a new cluster, which it may take several minutes to create.
  
1. When the endpoint has been created, use the **Query endpoint** button at the top right to open an interface from which you can test the endpoint. Then in the test interface, on the **Browser** tab, enter the following JSON request and use the **Send Request** button to call the endpoint and generate a prediction.

    ```json
    {
      "dataframe_records": [
      {
         "Island": "Biscoe",
         "CulmenLength": 48.7,
         "CulmenDepth": 14.1,
         "FlipperLength": 210,
         "BodyMass": 4450
      }
      ]
    }
    ```

1. Experiment with a few different values for the penguin features and observe the results that are returned. Then, close the test interface.

## Delete the endpoint

When the endpoint is not longer required, you should delete it to avoid unnecessary costs.

In the **predict-penguin** endpoint page, in the **&#8285;** menu, select **Delete**.